In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
from keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder

%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
data_exp=pd.read_csv('BikeSharing_Bluebikes2022.csv')
nat_hol = ['2022-01-01','2022-01-17','2022-02-21','2022-04-17','2022-04-18','2022-05-08','2022-05-30','2022-06-19','2022-06-20','2022-07-04']

data_exp['is_Holiday'] = [ 1 if data_exp.iloc[i]['starttime'][0:10] in nat_hol else 0 for i in range(len(data_exp))]
data_w = pd.read_csv("Boston 2022-01-01 to 2022-08-31.csv")
drop = ['dew', 'sunrise','sunset','moonphase','conditions', 'description','stations','sealevelpressure',
'windgust','severerisk', 'uvindex', 'solarenergy', 'solarradiation','preciptype','winddir','name']
data_w = data_w.drop(columns = drop)
data_w['datetime'] = pd.to_datetime(data_w['datetime'])

In [3]:
from datetime import datetime as dt

data_exp['starttime'] = pd.to_datetime(data_exp['starttime'], format='%Y-%m-%d %H:%M:%S')
data_exp['datetime'] = pd.to_datetime(data_exp['starttime']).dt.date
data_exp['datetime'] = pd.to_datetime(data_exp['datetime'])

data_merge = data_exp.merge(data_w, how='left', on='datetime')

In [4]:
data_merge.columns
data_merge = data_merge.drop(columns=(['Unnamed: 0']))

In [5]:
data_merge.columns


Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'postal code', 'is_Holiday', 'datetime', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip',
       'precipprob', 'precipcover', 'snow', 'snowdepth', 'windspeed',
       'cloudcover', 'visibility', 'icon'],
      dtype='object')

In [6]:
data_merge['start station id'].unique()

array([178, 189,  94,  19, 107,  36,  58,  60,   4, 377, 342, 491, 362,
        43, 121, 381,  10,  81, 332, 328, 233, 176,  55,  80,  12,  68,
        32, 202,  54,  74,  39,  66, 356, 116, 398,  42, 352, 330, 426,
        67, 225, 234,  91,  76, 161, 386, 374, 345, 437, 487,  40, 415,
       149, 183,  23,  71,   6,  72, 131, 118,  87,  86,  25,  97, 119,
       338,  69, 359, 446, 239,  22,  47,  26, 125,  33, 157, 105, 471,
        41, 174,  30,  16, 400, 177, 191, 152, 378, 218, 179, 403,  75,
       443, 379,  17,  46,   7,  11, 226, 363, 479, 190, 536, 141,  78,
       181, 139, 380, 417,  95,  90, 364, 549, 529,  44, 151, 376, 100,
        24, 137, 104, 389,  20,  96, 108,  98, 272, 187,  21, 541, 370,
        59, 393, 163, 413,  65, 215, 175, 414, 186, 318,  93, 344, 554,
       472,  63, 169, 196,  51, 419, 331,  84, 478, 515, 115, 213, 372,
       361, 192,  31, 136, 197, 138, 110, 273,  37,  49, 228, 456,  73,
       206,  70, 407, 194, 193, 440,   9, 480, 102, 327,   8,   

In [7]:
list = []
for i in data_merge['start station id'].unique():
    count = data_merge[data_merge['start station id']==i].count()[0]
    list.append(count)

maxim = data_merge['start station id'].unique()[list.index(max(list))]
print(maxim)
data_merge[data_merge['start station id']==maxim].count()[0]



67


49127

In [8]:
data_merge.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'postal code', 'is_Holiday', 'datetime', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip',
       'precipprob', 'precipcover', 'snow', 'snowdepth', 'windspeed',
       'cloudcover', 'visibility', 'icon'],
      dtype='object')

In [9]:
data_merge = data_merge[data_merge['start station id']==maxim]
data_nn = data_merge.drop(columns=['start station id',
       'start station name', 'start station latitude',
       'start station longitude','end station name',
       'end station latitude', 'end station longitude','bikeid','postal code', 'stoptime'])

In [10]:
data_nn

,tripduration,starttime,end station id,usertype,is_Holiday,datetime,tempmax,tempmin,temp,feelslikemax,...,humidity,precip,precipprob,precipcover,snow,snowdepth,windspeed,cloudcover,visibility,icon
62,2209,2022-01-01 00:29:15.682,67,Subscriber,1,2022-01-01,10.7,6.8,8.3,10.7,...,98.2,1.225,100,29.17,0.0,0.0,11.0,100.0,3.0,rain
75,510,2022-01-01 00:34:19.485,139,Subscriber,1,2022-01-01,10.7,6.8,8.3,10.7,...,98.2,1.225,100,29.17,0.0,0.0,11.0,100.0,3.0,rain
76,503,2022-01-01 00:34:25.574,139,Subscriber,1,2022-01-01,10.7,6.8,8.3,10.7,...,98.2,1.225,100,29.17,0.0,0.0,11.0,100.0,3.0,rain
83,606,2022-01-01 00:37:41.212,55,Subscriber,1,2022-01-01,10.7,6.8,8.3,10.7,...,98.2,1.225,100,29.17,0.0,0.0,11.0,100.0,3.0,rain
136,1452,2022-01-01 01:08:08.913,36,Subscriber,1,2022-01-01,10.7,6.8,8.3,10.7,...,98.2,1.225,100,29.17,0.0,0.0,11.0,100.0,3.0,rain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305598,1353,2022-08-31 23:54:24.025,96,Subscriber,0,2022-08-31,29.3,22.2,25.0,29.5,...,63.1,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain
2305608,1042,2022-08-31 23:54:38.824,143,Subscriber,0,2022-08-31,29.3,22.2,25.0,29.5,...,63.1,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain
2305623,1024,2022-08-31 23:55:04.391,143,Subscriber,0,2022-08-31,29.3,22.2,25.0,29.5,...,63.1,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain
2305661,346,2022-08-31 23:56:47.949,53,Subscriber,0,2022-08-31,29.3,22.2,25.0,29.5,...,63.1,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain


In [11]:
ran = data_nn['starttime'].iloc[3]
ran.hour
(data_nn['starttime'].iloc[0].hour >= 0) & (data_nn['starttime'].iloc[0].hour < 6)

True

In [12]:
range(len(data_nn))
# data_nn['tripduration'].iloc[0]=2
# data_nn['tripduration'].iloc[0]

range(0, 49127)

In [13]:
data_nn['time_of_day'] = 0
for i in range(len(data_nn)):
    if (data_nn['starttime'].iloc[i].hour >= 6) & (data_nn['starttime'].iloc[i].hour < 12):
        data_nn['time_of_day'].iloc[i] = 'morning'
    elif (data_nn['starttime'].iloc[i].hour >= 12) & (data_nn['starttime'].iloc[i].hour < 18):
        data_nn['time_of_day'].iloc[i] = 'afternoon'
    elif (data_nn['starttime'].iloc[i].hour >= 18) & (data_nn['starttime'].iloc[i].hour < 24):
        data_nn['time_of_day'].iloc[i] = 'evening'
    elif (data_nn['starttime'].iloc[i].hour >= 0) & (data_nn['starttime'].iloc[i].hour < 6):
        data_nn['time_of_day'].iloc[i] = 'night'


C:\Users\mirun\AppData\Local\Temp\ipykernel_6776\3777789447.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_nn['time_of_day'].iloc[i] = 'night'


In [14]:
data_nn = data_nn.drop(columns=['starttime','datetime'])

In [43]:
data_nn['end station id']

62          67
75         139
76         139
83          55
136         36
          ... 
2305598     96
2305608    143
2305623    143
2305661     53
2305730     84
Name: end station id, Length: 49127, dtype: int64

In [15]:
data_nn.columns

Index(['tripduration', 'end station id', 'usertype', 'is_Holiday', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth',
       'windspeed', 'cloudcover', 'visibility', 'icon', 'time_of_day'],
      dtype='object')

In [16]:
data_nn_dummies = pd.get_dummies(data_nn, columns=['time_of_day', 'icon', 'usertype'])

In [17]:
data_nn_dummies_std = data_nn_dummies.copy()

In [53]:
data_nn['end station id'].mode()


0    68
Name: end station id, dtype: int64

In [54]:

data_nn[data_nn['end station id']==68]

,tripduration,end station id,usertype,is_Holiday,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,...,precip,precipprob,precipcover,snow,snowdepth,windspeed,cloudcover,visibility,icon,time_of_day
588,1098,68,Subscriber,1,10.7,6.8,8.3,10.7,5.0,7.6,...,1.225,100,29.17,0.0,0.0,11.0,100.0,3.0,rain,night
2123,453,68,Subscriber,0,7.9,1.1,6.1,7.3,-3.5,3.6,...,5.814,100,50.00,0.0,0.0,27.5,95.1,7.3,rain,night
2907,343,68,Subscriber,0,7.9,1.1,6.1,7.3,-3.5,3.6,...,5.814,100,50.00,0.0,0.0,27.5,95.1,7.3,rain,afternoon
2955,440,68,Subscriber,0,7.9,1.1,6.1,7.3,-3.5,3.6,...,5.814,100,50.00,0.0,0.0,27.5,95.1,7.3,rain,afternoon
3305,325,68,Subscriber,0,7.9,1.1,6.1,7.3,-3.5,3.6,...,5.814,100,50.00,0.0,0.0,27.5,95.1,7.3,rain,evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303289,597,68,Subscriber,0,29.3,22.2,25.0,29.5,22.2,25.0,...,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain,evening
2304523,613,68,Subscriber,0,29.3,22.2,25.0,29.5,22.2,25.0,...,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain,evening
2304538,595,68,Subscriber,0,29.3,22.2,25.0,29.5,22.2,25.0,...,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain,evening
2304816,4142,68,Customer,0,29.3,22.2,25.0,29.5,22.2,25.0,...,8.351,100,16.67,0.0,0.0,31.6,49.3,16.0,rain,evening


0    68
Name: end station id, dtype: int64

In [18]:
col_std = ['tripduration', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth',
       'windspeed', 'cloudcover', 'visibility']

data_nn_dummies_std[col_std]= (data_nn_dummies_std[col_std] - data_nn_dummies_std[col_std].mean()) / data_nn_dummies_std[col_std].std()

In [19]:
data_nn_dummies_std

,tripduration,end station id,is_Holiday,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,humidity,...,time_of_day_evening,time_of_day_morning,time_of_day_night,icon_clear-day,icon_cloudy,icon_partly-cloudy-day,icon_rain,icon_snow,usertype_Customer,usertype_Subscriber
62,0.070575,67,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
75,-0.036324,139,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
76,-0.036764,139,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
83,-0.030284,55,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
136,0.022945,36,1,-0.919488,-0.462015,-0.741829,-0.780282,-0.359703,-0.587352,2.489714,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305598,0.016716,96,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1
2305608,-0.002851,143,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1
2305623,-0.003984,143,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1
2305661,-0.046643,53,0,0.952752,1.172287,1.029337,0.900253,1.093995,0.959238,0.045236,...,1,0,0,0,0,0,1,0,0,1


In [20]:
uniques = (data_nn_dummies_std['end station id'].unique()).tolist()
uniques.sort()
data_nn_dummies_std['categorical_target'] = [uniques.index(data_nn_dummies_std['end station id'].iloc[i]) for i in range(len(data_nn_dummies_std))]


In [21]:
data_nn_dummies_std = data_nn_dummies_std.drop(columns=['end station id'])

In [22]:
target = data_nn_dummies_std['categorical_target']
data_nn_dummies_std.set_index('categorical_target')
target.unique()

array([ 59, 125,  48,  31, 291,  98,  60, 150,  53,  67,  97, 232, 180,
        39, 105,  87,  81, 178, 236, 230,   9, 323, 309, 243,  68, 211,
        66, 216, 156,  38,  64,  95, 239, 223, 161,  88,  72, 157, 127,
       151,  85,  29,  75, 152,  79,   7, 258,  89, 149,  80, 208,  18,
        22, 209,  35, 261, 292,  47,  28, 240, 270, 324, 184,  36, 286,
       294,   4,   8, 162, 203, 234, 198,  14,  19, 226,  90,  33, 250,
       254, 106, 140, 165, 148, 265,  65, 100, 237, 249,   1, 129, 253,
        94, 181,  16,  10, 276,  77,  44, 185,  84, 205, 251,  58, 238,
       175,  73, 182, 327,  78, 206, 235, 248, 174, 119, 303,  32, 268,
       107, 326,  62, 213,  70, 186,  12, 167, 124,  37, 130,  42, 138,
       154, 204,  15,  69, 302,  34,  86, 120, 207, 101, 222,  92, 179,
        52,  51, 281,  61,  49,  13, 192, 153,  83, 147, 329, 301,  25,
       159,  40, 289,   2, 218, 221, 196, 212,  99, 314, 231,   6, 133,
        63, 319, 134,  24, 242, 278, 194, 287,  26, 228, 141, 26

In [64]:
target

62          59
75         125
76         125
83          48
136         31
          ... 
2305598     86
2305608    129
2305623    129
2305661     46
2305730     75
Name: categorical_target, Length: 49127, dtype: int64

In [23]:
split = int(len(data_nn_dummies_std)*0.6)
split_val = int(len(data_nn_dummies_std)*0.7)
x_train = data_nn_dummies_std[:split]
y_train = target[:split]
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
x_val = data_nn_dummies_std[split: split_val]
y_val = target[split: split_val]
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

x_test = data_nn_dummies_std[split_val:]
y_test = target[split_val:]
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))


C:\Users\mirun\AppData\Local\Temp\ipykernel_6776\2643619492.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = target[:split]
C:\Users\mirun\AppData\Local\Temp\ipykernel_6776\2643619492.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_val = target[split: split_val]
C:\Users\mirun\AppData\Local\Temp\ipykernel_6776\2643619492.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *l

In [24]:
print(len(x_train), len(x_val) ,len(x_test),len(x_train)+len(x_val)+len(x_test) == len(data_nn_dummies_std))

29476 4912 14739 True


In [25]:
num_features = x_train.shape[1]
num_output = len(target.unique())
num_output


337

In [26]:
x_train.shape[1]

29

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# define the keras model
model = Sequential()
model.add(Dense(800, input_shape = (x_train.shape[1],), activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(600, activation='relu'))
# model.add(Dropout(rate=0.5))
model.add(Dense(500, activation='relu'))
# model.add(Dropout(rate=0.5))
model.add(Dense(num_output, activation='softmax'))

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=50, batch_size=500, validation_data=(x_val,y_val))



Epoch 1/50
59/59 [==============================] - 1s 15ms/step - loss: 5.4960 - accuracy: 0.0799 - val_loss: 4.7873 - val_accuracy: 0.0664
Epoch 2/50
59/59 [==============================] - 1s 13ms/step - loss: 3.6596 - accuracy: 0.1756 - val_loss: 3.6329 - val_accuracy: 0.1737
Epoch 3/50
59/59 [==============================] - 1s 13ms/step - loss: 2.9986 - accuracy: 0.2830 - val_loss: 3.1664 - val_accuracy: 0.2437
Epoch 4/50
59/59 [==============================] - 1s 13ms/step - loss: 2.6260 - accuracy: 0.3354 - val_loss: 2.8452 - val_accuracy: 0.2915
Epoch 5/50
59/59 [==============================] - 1s 12ms/step - loss: 2.4299 - accuracy: 0.3570 - val_loss: 2.7197 - val_accuracy: 0.3090
Epoch 6/50
59/59 [==============================] - 1s 14ms/step - loss: 2.2689 - accuracy: 0.3776 - val_loss: 2.5590 - val_accuracy: 0.3528
Epoch 7/50
59/59 [==============================] - 1s 13ms/step - loss: 2.1252 - accuracy: 0.4021 - val_loss: 2.4677 - val_accuracy: 0.3414
Epoch 8/50
59

In [44]:
prediction = model.predict(x_test)

461/461 [==============================] - 1s 2ms/step


In [45]:
loss, accuracy =  model.evaluate(x_test, y_test, verbose=2)

print("Accuracy   : {:5.2f} ".format(accuracy))

461/461 - 1s - loss: 2.0958 - accuracy: 0.5993 - 1s/epoch - 3ms/step
Accuracy   :  0.60 


In [47]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score

# function to evaluate predictions
def evaluate(y_true, y_pred):
    # calculate and display confusion matrix
    labels = np.unique(y_true)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print('Confusion matrix\n- x-axis is true labels \n- y-axis is predicted labels')
    print(cm)

    # calculate precision, recall, and F1 score
    accuracy = float(np.trace(cm)) / np.sum(cm)
    precision = precision_score(y_true, y_pred, average=None, labels=labels)[1]
    recall = recall_score(y_true, y_pred, average=None, labels=labels)[1]
    f1 = 2 * precision * recall / (precision + recall)
    print("accuracy:", accuracy)
    print("precision:", precision)
    print("recall:", recall)
    print("f1 score:", f1)
    
# evaluate the keras model
y_pred = model.predict(x_test)
# y_pred = (y_pred > 0.5).astype(int)

# evaluate predictions
print("- Test set results:")
evaluate(y_test, y_pred)

461/461 [==============================] - 1s 3ms/step
- Test set results:


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [72]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction, average=None)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [75]:
y_test.shape

(14739,)

In [74]:
prediction.shape

(14739, 337)

In [60]:
import numpy as np
y_test = y_test.astype(int)

## Random tree forest classifier


In [31]:
from sklearn.tree import DecisionTreeClassifier


In [33]:
dt=DecisionTreeClassifier()
dt.fit(x_train, y_train)
dt.score(x_test,y_test)


0.986159169550173

In [36]:
y_test

array([[  5.],
       [ 29.],
       [323.],
       ...,
       [129.],
       [ 46.],
       [ 75.]], dtype=float32)

In [38]:
ypred=dt.predict(x_test)
print(confusion_matrix(y_test, ypred))

[[ 0  3  0 ...  0  0  0]
 [ 0 20  0 ...  0  0  0]
 [ 0  0 18 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
